# ETL Helper
> A combined ETL tool sets

In [1]:
# default_exp etl

## A list for each step

In [23]:
# export

from forgebox.imports import *
from typing import Callable, Union
import traceback as tb
import logging

class NewFileScanner:
    """
    Keep scannning a directory for new file
    if found any, callback processing the file
    
    Designed for download and delete strategy
    
    # Example
    new_file_scanner = NewFileScanner(".", new_file_filter=lambda x:x[-4:]==".txt")
    new_file_scanner(lambda x:print(f"new file:{x} found"))s
    """
    def __init__(
        self,
        directory,
        new_file_filter: Callable=None,
        done_list_getter: Callable=None,
        stop_callback: Callable=None,
    ):
        self.directory = Path(directory)
        
        if new_file_filter is not None:
            self.new_file_filter=new_file_filter
        else:
            self.new_file_filter=lambda x:True
        
        if done_list_getter is not None:
            self.done_list_getter = done_list_getter
        else:
            self.done_list_getter = lambda *args:[]
            
        if stop_callback is not None:
            self.stop_callback = stop_callback
            
        self.processed = []
        
    def __call__(self, process: Callable):
        while True:
            try:
                done_list = self.done_list_getter()
                for fname in self.directory.iterdir():
                    fname = str(fname)
                    if self.new_file_filter(fname) != True:
                        continue
                    file_path = str(self.directory/fname)
                    if fname not in done_list:
                        if fname not in self.processed:
                            result = process(file_path)
                            self.processed.append(fname)
            except KeyboardInterrupt as e:
                if hasattr(self, "stop_callback"):
                    self.stop_callback()
                logging.error("manually stoped")
                break
            except Exception as e:
                error = tb.format_exc()
                logging.error(error)

### Test new file scanner

In [ ]:
new_file_scanner = NewFileScanner(".", new_file_filter=lambda x:x[-4:]==".txt")
new_file_scanner(lambda x:print(f"new file:{x} found"))

new file:untitled.txt found
new file:bc.txt found
new file:cde.txt found


ERROR:root:manually stoped
